# Disclaimer:
DeepSphere.AI developed these
materials based on its team’s expertise
and technical infrastructure, and we
are sharing these materials strictly for
learning and research. These learning
resources may not work on other learning
infrastructures and DeepSphere.AI
advises the learners to use these materials
at their own risk. As needed, we will be
changing these materials without any
notification and we have full ownership
and accountability to make any change
to these materials.

# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Libraries and Install Dependencies

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Class ChatBot

In [3]:
class ChatBot:

  def __init__(self, intents): #constructor
  #intents - data from json file
    self.words=[]
    self.classes = []
    self.documents = []
    self.ignore_words = ['?', '!']
    self.intents = intents
    self.lemmatizer = WordNetLemmatizer()
    
    for intent in self.intents['intents']:
      for pattern in intent['patterns']:

        vAR_w = nltk.word_tokenize(pattern) # take each word and tokenize it
        self.words.extend(vAR_w)
        self.documents.append((vAR_w, intent['tag']))  # adding documents

        if intent['tag'] not in self.classes: # adding classes to our class list
            self.classes.append(intent['tag'])

    self.words = [self.lemmatizer.lemmatize(w.lower()) for w in self.words if w not in self.ignore_words]
    self.words = sorted(list(set(self.words)))
    self.classes = sorted(list(set(self.classes)))

    print(len(self.documents), "documents")
    print(len(self.classes), "classes", self.classes)
    print(len(self.words), "unique lemmatized words", self.words)

  def store_object(self): #store/dump words and classes as pickle file
    pickle.dump(self.words,open('DSAI_words.pkl','wb'))
    pickle.dump(self.classes,open('DSAI_classes.pkl','wb'))

  def initialize_training(self):
    # initializing training data
    self.training = []
    vAR_output_empty = [0] * len(self.classes)

    for doc in self.documents:
        vAR_bag = []  # initializing bag of words
        vAR_pattern_words = doc[0]  # list of tokenized words for the pattern
        vAR_pattern_words = [self.lemmatizer.lemmatize(word.lower()) for word in vAR_pattern_words] # lemmatize each word - create base word, in attempt to represent related words

        # create our bag of words array with 1, if word match found in current pattern
        for w in self.words:
            vAR_bag.append(1) if w in vAR_pattern_words else vAR_bag.append(0)

        # output is a '0' for each tag and '1' for current tag (for each pattern)
        vAR_output_row = list(vAR_output_empty)
        vAR_output_row[self.classes.index(doc[1])] = 1

        self.training.append([vAR_bag, vAR_output_row])
    
    random.shuffle(self.training)  # shuffle our features and turn into np.array
    self.training = np.array(self.training)

  def create_train_test_lists(self):
    # create train and test lists. X - patterns, Y - intents
    self.train_x = list(self.training[:,0])
    self.train_y = list(self.training[:,1])
    print("Training data created")

  def create_model(self):
    # Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
    # equal to number of intents to predict output intent with softmax
    self.model = Sequential()
    self.model.add(Dense(128, input_shape=(len(self.train_x[0]),), activation='relu'))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(64, activation='relu'))
    self.model.add(Dropout(0.5))
    self.model.add(Dense(len(self.train_y[0]), activation='softmax'))

  def compile_model(self):
    # Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
    vAR_sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    self.model.compile(loss='categorical_crossentropy', optimizer = vAR_sgd, metrics=['accuracy'])

  def fit_and_save_model(self):
    #fitting and saving the model
    vAR_hist = self.model.fit(np.array(self.train_x), np.array(self.train_y), epochs=200, batch_size=5, verbose=1)
    self.model.save('DSAI_chatbot_model.h5', vAR_hist)

    print("model created")

# Import Data

In [4]:
data_file = open('/content/drive/MyDrive/26-Industry Use Case-Retail-Chatbot_keras/DSAI_intents.json').read()
intents = json.loads(data_file)

# Create Object

In [5]:
object = ChatBot(intents)

46 documents
20 classes ['Textbook', 'access', 'ask', 'askorder', 'benefits', 'career', 'complaint', 'courses', 'createaccount', 'demo', 'goodbye', 'greeting', 'help', 'levels', 'name', 'pay', 'prerequisites', 'registration', 'thanks', 'time']
86 unique lemmatized words ["'s", '.', 'a', 'about', 'access', 'account', 'an', 'anyone', 'are', 'at', 'before', 'benefit', 'buy', 'bye', 'call', 'can', 'career', 'certain', 'chapter', 'complaint', 'could', 'course', 'create', 'data', 'demo', 'do', 'doing', 'enroll', 'enrollment', 'for', 'give', 'goodbye', 'hand', 'have', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'in', 'is', 'it', 'later', 'level', 'me', 'meet', 'my', 'name', 'need', 'new', 'offered', 'offering', 'online', 'open', 'order', 'pay', 'please', 'prerequisite', 'raise', 'register', 'registered', 'science', 'see', 'service', 'should', 'specific', 'start', 'support', 'take', 'textbook', 'thank', 'thanks', 'that', 'the', 'there', 'time', 'to', 'want', 'what', 'whats', 'w

# Store Object as Pickle Format

In [6]:
object.store_object()

# Initialize Training Data

In [7]:
object.initialize_training()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


# Create train and test lists

In [8]:
object.create_train_test_lists()

Training data created


# Model Creation

In [9]:
object.create_model()

# Compile Model

In [10]:
object.compile_model()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


# Fit and Save Model

In [11]:
object.fit_and_save_model()

Epoch 1/200
10/10 [==============================] - 1s 5ms/step - loss: 2.9682 - accuracy: 0.0652
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.9055 - accuracy: 0.1522
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.7502 - accuracy: 0.1957
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.7408 - accuracy: 0.1957
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 2.6187 - accuracy: 0.1522
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 2.5846 - accuracy: 0.2826
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 2.4918 - accuracy: 0.3478
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.3249 - accuracy: 0.3043
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1881 - accuracy: 0.2609
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1270 - accuracy: 0.4348
Epoch 11/

# Copyright Notice:
Local and international copyright laws protect
this material. Repurposing or reproducing
this material without written approval from
DeepSphere.AI violates the law.

(c) DeepSphere.AI